# Diabetes Machine Learning Group Project

Data Tables from 2017-18 from NHANES
- Diabetes (DIQ_J) https://wwwn.cdc.gov/Nchs/Nhanes/2017-2018/DIQ_J.htm
- Health Insurance (HIQ_J) https://wwwn.cdc.gov/Nchs/Nhanes/2017-2018/HIQ_J.htm
- Food Security (FSQ_J) https://wwwn.cdc.gov/Nchs/Nhanes/2017-2018/FSQ_J.htm
- Income (INQ_J) https://wwwn.cdc.gov/Nchs/Nhanes/2017-2018/INQ_J.htm
- Blood Pressure (BPX_J) https://wwwn.cdc.gov/Nchs/Nhanes/2017-2018/BPX_J.htm
- Body Measures (BMX_J) https://wwwn.cdc.gov/Nchs/Nhanes/2017-2018/BMX_J.htm

8,897 Original Respondents

## Merging

In [49]:
# Load packages
import pandas as pd

In [50]:
# Load in tables
diq = pd.read_csv("./resources/DIQ_J_2017-18.csv", index_col = 0)
fsq = pd.read_csv("./resources/FSQ_J_2017-18.csv", index_col = 0)
inq = pd.read_csv("./resources/INQ_J_2017-18.csv", index_col = 0)
hiq = pd.read_csv("./resources/HIQ_J_2017-18.csv", index_col = 0)
bpx = pd.read_csv("./resources/BPX_J_2017-18.csv", index_col = 0)
bmx = pd.read_csv("./resources/BMX_J_2017-18.csv", index_col = 0)

In [51]:
# Generate merged df of everything
merged = pd.merge(diq, fsq, on = "SEQN")
merged = pd.merge(merged, inq, on = "SEQN")
merged = pd.merge(merged, hiq, on = "SEQN")
merged = pd.merge(merged, bpx, on = "SEQN")
merged = pd.merge(merged, bmx, on = "SEQN")
merged.head()

,SEQN,DIQ010,DID040,DIQ160,DIQ170,DIQ172,DIQ175A,DIQ175B,DIQ175C,DIQ175D,...,BMXLEG,BMILEG,BMXARML,BMIARML,BMXARMC,BMIARMC,BMXWAIST,BMIWAIST,BMXHIP,BMIHIP
0,93703.0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,18.0,NaN,16.2,NaN,48.2,NaN,NaN,NaN
1,93704.0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,18.6,NaN,15.2,NaN,50.0,NaN,NaN,NaN
2,93705.0,2,NaN,2.0,2.0,2.0,NaN,NaN,NaN,NaN,...,37.0,NaN,36.0,NaN,32.0,NaN,101.8,NaN,110.0,NaN
3,93706.0,2,NaN,2.0,2.0,2.0,NaN,NaN,NaN,NaN,...,46.6,NaN,38.8,NaN,27.0,NaN,79.3,NaN,94.4,NaN
4,93707.0,2,NaN,2.0,2.0,2.0,NaN,NaN,NaN,NaN,...,38.1,NaN,33.8,NaN,21.5,NaN,64.1,NaN,83.0,NaN


In [52]:
# Export merged dataframe
merged.to_csv("./resources/merged.csv")

## Selection, Filtering, and Formatting

In [43]:
# Convert diabetic columns to intuitive labels


(8897, 132)